In [1]:
import pandas as pd
from fast_pandas import FastPandas

In [2]:
df = pd.DataFrame({"a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], "b": [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]})

In [9]:
# lazy eval
out = FastPandas(df)["a"].avg()
print(out)

# nothing has been computed yet!
# now when you access the item, it will be computed
print(out.item())

avg("a")
5.5


In [5]:
import random
# you can choose from the aggregate or numeric functions found on DuckDB
large_df = pd.DataFrame({"a": random.choices(range(35_000_000), k=100_000_000)})

In [6]:
unique_elements = FastPandas(large_df)["a"].approx_count_distinct().item()
print(unique_elements) # uses hyperloglog under the hood

33028986


In [7]:
# that was on a 100 million row dataframe in much less than a second!!!
print(f"{len(large_df):,}")

100,000,000


In [8]:
# correlation between two columns for 100 million rows???
df["b"] = a.abs().add(1).ln().df() # some random other column

# a is y, b is x
print(FastPandas(df)["a"].corr(FastPandas(df)["b"]).item())

0.9725752323144956


In [11]:
df = pd.DataFrame({'a': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'b': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]})

output = FastPandas(df)["a"].mul(FastPandas(df)["b"]).abs().add(1).entropy()
print(output.item())

2.321928094887362
